# 1. Decontextualize Tables
We want to do this by generating glossaries for each entry in the table

In [2]:
import json
import re
import requests

import pandas as pd

In [14]:
# with open("../data/arxiv_tables_2308_high_quality/dataset.jsonl") as f:
with open("../data/arxiv_tables_2308_high_quality/dataset_autocontext.jsonl") as f:
    dataset_2308_high_quality_sample = [json.loads(line.strip()) for line in f]

In [3]:
with open("../data/arxiv_tables_2308_high_quality/full_texts_with_tables_v2.jsonl") as f:
    full_texts_2308_high_quality = [json.loads(line) for line in f]
    full_texts_2308_high_quality_map = {ft["paper_id"]: ft for ft in full_texts_2308_high_quality}

{'paper_id': '2308.00729v1',
 '_pdf_hash': None,
 '_source_hash': 'add89d24da98870765fefba429d689dc4a83485c',
 '_source_name': '2308.00729v1',
 'metadata': {},
 'abstract': {'section': 'Abstract',
  'text': '',
  'cite_spans': [],
  'ref_spans': []},
 'ref_entries': {},
 'tables': {'43c9402e-2914-42dc-99c4-b2424f71f6ad': {'table': '<table rend="display" id-text="1" id="uid1" place="t"><head>Comparisons of public VQA and video classification datasets. Note that obtaining reliable quality annotations requires massive subjective annotations per video, which limits the size of VQA datasets.</head>\n<row><cell right-border="true" halign="center">Dataset</cell>\n<cell right-border="true" halign="center">Task</cell>\n<cell halign="center">Size</cell>\n<cell>Annotations</cell>\n</row><row><cell right-border="true" halign="center">KoNViD-1k <cit sha="24b4d04b01098cffe3cb975171aa05132c6a0903"><ref target="bid14"/></cit>{{cite:24b4d04}}</cell>\n<cell right-border="true" halign="center">VQA</cell>

In [10]:
with open("../data/arxiv_tables_2308_high_quality/papers_expanded.jsonl") as f:
    papers_2308_high_quality = [json.loads(line) for line in f]

In [11]:
metadata_map = {paper['arxiv_id']: paper for paper in papers_2308_high_quality if 'arxiv_id' in paper}

In [12]:
metadata_map

{'2308.00729': {'tabids': [],
  'corpus_id': 260378902,
  'arxiv_id': '2308.00729',
  'paper_id': 'b5e4c46ac09d5de2051998c9f741ef480fab2ebe',
  'title': 'Ada-DQA: Adaptive Diverse Quality-aware Feature Acquisition for Video Quality Assessment',
  'abstract': 'Video quality assessment (VQA) has attracted growing attention in recent years. While the great expense of annotating large-scale VQA datasets has become the main obstacle for current deep-learning methods. To surmount the constraint of insufficient training data, in this paper, we first consider the complete range of video distribution diversity (i.e. content, distortion, motion) and employ diverse pretrained models (e.g. architecture, pretext task, pre-training dataset) to benefit quality representation. An Adaptive Diverse Quality-aware feature Acquisition (Ada-DQA) framework is proposed to capture desired quality-related features generated by these frozen pretrained models. By leveraging the Quality-aware Acquisition Module (Q

In [14]:
# for i in range(len(full_texts_2308_high_quality)):
#     full_texts_2308_high_quality[i] = {k:, 

for ft in full_texts_2308_high_quality:
    arxiv_id_no_version = re.sub("v\d+", "", ft['paper_id'])
    ft['title'] = metadata_map[arxiv_id_no_version]['title']
    ft['abstract']['text'] = metadata_map[arxiv_id_no_version]['abstract']

In [16]:
with open("../data/arxiv_tables_2308_high_quality/full_texts_with_tables.jsonl", "w") as f:
    for full_text in full_texts_2308_high_quality:
        f.write(json.dumps(full_text) + "\n")
    # full_texts_2308_high_quality = [json.loads(line) for line in f]
    # full_texts_2308_high_quality_map = {ft["paper_id"]: ft for ft in full_texts_2308_high_quality}

In [15]:
full_texts_2308_high_quality[0]

{'paper_id': '2308.00729v1',
 '_pdf_hash': None,
 '_source_hash': 'add89d24da98870765fefba429d689dc4a83485c',
 '_source_name': '2308.00729v1',
 'metadata': {},
 'abstract': {'section': 'Abstract',
  'text': 'Video quality assessment (VQA) has attracted growing attention in recent years. While the great expense of annotating large-scale VQA datasets has become the main obstacle for current deep-learning methods. To surmount the constraint of insufficient training data, in this paper, we first consider the complete range of video distribution diversity (i.e. content, distortion, motion) and employ diverse pretrained models (e.g. architecture, pretext task, pre-training dataset) to benefit quality representation. An Adaptive Diverse Quality-aware feature Acquisition (Ada-DQA) framework is proposed to capture desired quality-related features generated by these frozen pretrained models. By leveraging the Quality-aware Acquisition Module (QAM), the framework is able to extract more essential

In [364]:
# with open("../data/arxiv_tables_2308_high_quality/papers_expanded.jsonl", "w") as f:
#     for line in papers_2308_high_quality:
#         f.write(json.dumps(line) + "\n")
    # papers_2308_high_quality = [json.loads(line) for line in f]

The next step is actually to get the context for the table from the full-text. There are a few pieces of this:
1. The caption: this is easiest because it's included in the info extracted from the table
2. The table mention: this is next easiest because we can just see where the table is referenced
3. The glossary: this is hard because it involves some kind of paperqa. It is also the most important which is kind of unfortunate... I can use my gpt pipeline, but because we need to decontextualize *every* table entry, that's gonna be kinda expensive. What are our options?
    1. Use mistral? (good to try, but I would need to move over some data to s2 cluster, etc.)
    2. DSPY? not enough in-context examples probably
    3. Exact string match? eh quite brittle


In [361]:
papers_2308_high_quality[0]

{'tabids': ['b42af37c-c83f-4cc0-8cc8-e7221c6a8d0f'],
 'corpus_id': 9136312,
 'title': 'The Konstanz natural video database (KoNViD-1k)',
 'paper_id': 'ae95abd2406ddfab1aa2ffa2413d68b98b1ba21b',
 'abstract': 'Subjective video quality assessment (VQA) strongly depends on semantics, context, and the types of visual distortions. Currently, all existing VQA databases include only a small number of video sequences with artificial distortions. The development and evaluation of objective quality assessment methods would benefit from having larger datasets of real-world video sequences with corresponding subjective mean opinion scores (MOS), in particular for deep learning purposes. In addition, the training and validation of any VQA method intended to be ‘general purpose’ requires a large dataset of video sequences that are representative of the whole spectrum of available video content and all types of distortions. We report our work on KoNViD-1k, a subjectively annotated VQA database consist

In [103]:
# to start, let's see how we would do this with the first table:
full_text = full_texts_2308_high_quality_map[dataset_2308_high_quality_sample[0]['paper_id']]


In [ ]:

('Inference Time', '29min22s')
('Inference Time', '21min56s')
('Inference Time', '9s')

('Baseband BW', '1 GHz')
('Baseband BW', '4 GHz')
('Baseband BW', '2 GHz')
('Baseband BW', '450 MHz')
('Baseband BW', '7 GHz')
('Carries Freq.', 'Carries Freq.')
('Carries Freq.', '57 ∼ 64 GHz')
('Carries Freq.', '60 GHz')
('Carries Freq.', '24 GHz')
('Cost', '$15K')
('Cost', 'below $15K')
('Cost', '$40K')
('Cost', 'below $100')
('Video Training', '∼ 10 hr')
('Video Training', '∼ -')
('Edit Training', '∼ 1 hours')
('Edit Training', '30 min')
('Edit Inference', '∼ 10 sec')
('Edit Inference', '∼ 4 min')
('Edit Inference', '∼ 30 sec')

In [230]:
re.match?

Signature: re.match(pattern, string, flags=0)
Docstring:
Try to apply the pattern at the start of the string, returning
a Match object, or None if no match was found.
File:      /opt/conda/envs/jupyter/lib/python3.9/re.py
Type:      function

In [248]:
re.sub(r"[∼∼\s]", "", '∼ 10 hr'.lower().strip())

'10hr'

In [299]:
is_numeric('Carries Freq.')

carriesfreq.
carriesfreq.
False


False

In [273]:
any((None, None, None))

False

In [255]:
is_numeric('below $100')

below$100


True

In [276]:
is_numeric('∼ 10 hr')

10hr
10hr
None
('10hr', None, None)
None


True

In [286]:
re.match?

Signature: re.match(pattern, string, flags=0)
Docstring:
Try to apply the pattern at the start of the string, returning
a Match object, or None if no match was found.
File:      /opt/conda/envs/jupyter/lib/python3.9/re.py
Type:      function

In [287]:
False or False or False

False

In [ ]:
'below $15K', '–', '×', '∼ -'

In [318]:
is_na('∼ -')

True

In [311]:
is_numeric('below $15K')

15k
15k


True

In [7]:
def is_numeric(value):
    # value = value.lower()
    value = value.replace("below", "<")
    value = re.sub(r"[<$∼~∼\s]", "", value.lower().strip())
    # print(value)
    units_regex = r"[<-]?\d+[km]"
    time_regex = r"(\d+(?:hrs?|hours?))?(\d+(?:ms?|mins?|minutes?))?(\d+(?:s|sec|seconds?))?"
    freq_regex = r"\d+[gm]hz"
    # print(value)
    # print(re.match(units_regex, value) is not None or any(re.match(time_regex, value).groups()) or re.match(freq_regex, value) is not None)
    if re.match(units_regex, value) is not None or any(re.match(time_regex, value).groups()) or re.match(freq_regex, value) is not None:
        # print(re.match(units_regex, value) is not None)
        # print(any(re.match(time_regex, value).groups()))
        # print(re.match(freq_regex, value) is not None)
        return True
    # this is dumb, but easy and fast
    try:
        float(value.replace(",", "").replace("-", ""))
        return True
    except ValueError:
        return False

def is_binary(value):
    value = value.strip()
    return value.lower() in {'yes', 'no'} or value in {'✘', '✗', '×', '✔', '✓'}

def is_na(value):
    value = re.sub("[∼~\s]", "", value.lower())
    return value in {"-", '–', '-'}

In [176]:
glossary_keys = [
    (scheme.strip(), entry.strip())
    for scheme, column in
    dataset_2308_high_quality_sample[0]['table_json']['table_dict'].items()
    for entry in [scheme] + column
    if scheme != "References" and not is_numeric(str(entry)) and not is_na(str(entry)) and not is_binary(str(entry))
]

In [177]:
glossary_keys

[('Dataset', 'Dataset'),
 ('Dataset', 'KoNViD-1k'),
 ('Dataset', 'LIVE-VQC'),
 ('Dataset', 'YouTube-UGC'),
 ('Dataset', 'LSVQ'),
 ('Dataset', 'KoNViD-150k'),
 ('Dataset', 'Sports-1M'),
 ('Dataset', 'Kinetics-400'),
 ('Task', 'Task'),
 ('Task', 'VQA'),
 ('Task', 'VQA'),
 ('Task', 'VQA'),
 ('Task', 'VQA'),
 ('Task', 'VQA'),
 ('Task', 'classification'),
 ('Task', 'classification'),
 ('Size', 'Size'),
 ('Annotations', 'Annotations'),
 ('Annotations', '- (auto.)')]

In [55]:
for scheme, glossary_key in glossary_keys:
    if scheme == glossary_key:
        query = f"In the context of the following table, what does '{glossary_key}' refer to?\n"
    else:
        query = f"In the context of the following table, what does the '{scheme}' '{glossary_key}' refer to?"
    table = str(pd.DataFrame(dataset_2308_high_quality_sample[0]['table_json']['table_dict'])) + "\n"
    table += "Caption: " + dataset_2308_high_quality_sample[0]['table_unfiltered']['caption']
    break

In [56]:
print(query)

In the context of the following table, what does the 'Dataset' 'KoNViD-1k' refer to?


In [75]:
full_text['body_text']

[{'section': 'Introduction',
  'sec_number': '1',
  'sec_type': 'section',
  'content_type': 'paragraph',
  'text': "With the explosive growth of video content-based social media, there has been a tremendous amount of videos produced and shared {{cite:cef100d}}. To guarantee optimal video quality and ensure users' quality of experience (QoE), VQA plays a crucial role in guiding image processing and video coding systems {{cite:9b70eb4}}, {{cite:c6c0872}}. Benefit from the thrive of Deep Neural Networks (DNN), DNN-based VQA methods {{cite:7da4179}}, {{cite:1ad644b}}, {{cite:3dfd446}}, {{cite:9c04ab9}}, {{cite:5d0c0fc}}, {{cite:3237048}} have shown great results on in-the-wild VQA benchmarks {{cite:8f3eb86}}, {{cite:24b4d04}}, {{cite:9be8207}}, {{cite:3dfd446}}.\n",
  'cite_spans': [{'start': 128,
    'end': 144,
    'text': '{{cite:cef100d}}',
    'ref_id': 'cef100d'},
   {'start': 306, 'end': 322, 'text': '{{cite:9b70eb4}}', 'ref_id': '9b70eb4'},
   {'start': 324, 'end': 340, 'text': '{

In [92]:
full_text['paper_id']

'2308.00729v1'

In [97]:
for paragraph in full_text['body_text']:
    print(paragraph['text'])
    print()

With the explosive growth of video content-based social media, there has been a tremendous amount of videos produced and shared {{cite:cef100d}}. To guarantee optimal video quality and ensure users' quality of experience (QoE), VQA plays a crucial role in guiding image processing and video coding systems {{cite:9b70eb4}}, {{cite:c6c0872}}. Benefit from the thrive of Deep Neural Networks (DNN), DNN-based VQA methods {{cite:7da4179}}, {{cite:1ad644b}}, {{cite:3dfd446}}, {{cite:9c04ab9}}, {{cite:5d0c0fc}}, {{cite:3237048}} have shown great results on in-the-wild VQA benchmarks {{cite:8f3eb86}}, {{cite:24b4d04}}, {{cite:9be8207}}, {{cite:3dfd446}}.


DNN-based VQA methods are hindered by the limited scale of existing VQA datasets. As demonstrated in Tab. REF , public VQA datasets are significantly smaller in size when compared to video classification datasets {{cite:fbaab96}}, {{cite:dd352e3}}. The primary reason for this discrepancy is the prevalent use of crowdsourcing to annotate subjec

In [98]:
full_text['body_text'][1]

{'section': 'Introduction',
 'sec_number': '1',
 'sec_type': 'section',
 'content_type': 'paragraph',
 'text': 'DNN-based VQA methods are hindered by the limited scale of existing VQA datasets. As demonstrated in Tab.\xa0REF , public VQA datasets are significantly smaller in size when compared to video classification datasets {{cite:fbaab96}}, {{cite:dd352e3}}. The primary reason for this discrepancy is the prevalent use of crowdsourcing to annotate subjective video quality {{cite:b5fb90b}}, {{cite:2222c26}}, {{cite:689f656}}, {{cite:11e5a31}}, which is a time-consuming yet indispensable process in eliminating randomness and enhancing consistency. For example, the KoNViD-1k dataset {{cite:24b4d04}} requires an average of 114 subjective scores to produce a valid label. Recently, there have been attempts to scale up VQA datasets {{cite:3dfd446}}, {{cite:9c04ab9}}. However, these efforts often sacrifice the annotation quality, as they reduce the average number of annotations per video in 

In [99]:
for paragraph in full_text['body_text']:
    # for ref_span in paragraph['ref_spans']:
    if "DNN-based VQA method" in paragraph["text"]:
        print(paragraph)

{'section': 'Introduction', 'sec_number': '1', 'sec_type': 'section', 'content_type': 'paragraph', 'text': "With the explosive growth of video content-based social media, there has been a tremendous amount of videos produced and shared {{cite:cef100d}}. To guarantee optimal video quality and ensure users' quality of experience (QoE), VQA plays a crucial role in guiding image processing and video coding systems {{cite:9b70eb4}}, {{cite:c6c0872}}. Benefit from the thrive of Deep Neural Networks (DNN), DNN-based VQA methods {{cite:7da4179}}, {{cite:1ad644b}}, {{cite:3dfd446}}, {{cite:9c04ab9}}, {{cite:5d0c0fc}}, {{cite:3237048}} have shown great results on in-the-wild VQA benchmarks {{cite:8f3eb86}}, {{cite:24b4d04}}, {{cite:9be8207}}, {{cite:3dfd446}}.\n", 'cite_spans': [{'start': 128, 'end': 144, 'text': '{{cite:cef100d}}', 'ref_id': 'cef100d'}, {'start': 306, 'end': 322, 'text': '{{cite:9b70eb4}}', 'ref_id': '9b70eb4'}, {'start': 324, 'end': 340, 'text': '{{cite:c6c0872}}', 'ref_id': '

In [134]:
# as a baseline, let's use title, abstract, and section where the table comes from as input
section_names = []
for paragraph in full_text['body_text']:
    for ref_span in paragraph['ref_spans']:
        # if "DNN-based VQA method" in paragraph["body_text"]:
        #     print(paragraph)
        if ref_span["ref_id"] == dataset_2308_high_quality_sample[0]['_full_text_table_hash']:
            # print(paragraph['content_type'])
            # print(paragraph.keys())
            section_names.append(paragraph['section'])


def listify(list_paragraph):
    return " * " + "\n * ".join(list_paragraph.strip().split("\n\n"))

relevant_sections = {sn: [] for sn in section_names}
for paragraph in full_text['body_text']:
    if paragraph['section'] in relevant_sections:
        if paragraph['content_type'] == "paragraph":
            relevant_sections[paragraph['section']].append(paragraph['text'])
        elif paragraph['content_type'] == "list":
            relevant_sections[paragraph['section']].append(listify(paragraph['text']))
        else:
            print(paragraph['content_type'])

table


In [133]:
print(" * " + "\n * ".join(relevant_sections['Introduction'][-1].strip().split("\n\n")))

 * To the best of our knowledge, this is the first study to comprehensively investigate the relationships between pretrained models and video quality. We construct a diverse pool that encompasses a broad spectrum of quality-related factors.
 * We propose the Ada-DQA to leverage these pretrained models for VQA, where the QAM is proposed to capture quality-related features adaptively. Additionally, a sparsity constraint is also attached for the most crucial and relevant features.
 * We evaluate Ada-DQA on three mainstream NR-VQA benchmarks, surpassing current state-of-the-art methods without using extra training data of QA. Sufficient ablation studies validated the effectiveness of each component.


In [135]:
print("\n".join(relevant_sections['Introduction']))

With the explosive growth of video content-based social media, there has been a tremendous amount of videos produced and shared {{cite:cef100d}}. To guarantee optimal video quality and ensure users' quality of experience (QoE), VQA plays a crucial role in guiding image processing and video coding systems {{cite:9b70eb4}}, {{cite:c6c0872}}. Benefit from the thrive of Deep Neural Networks (DNN), DNN-based VQA methods {{cite:7da4179}}, {{cite:1ad644b}}, {{cite:3dfd446}}, {{cite:9c04ab9}}, {{cite:5d0c0fc}}, {{cite:3237048}} have shown great results on in-the-wild VQA benchmarks {{cite:8f3eb86}}, {{cite:24b4d04}}, {{cite:9be8207}}, {{cite:3dfd446}}.

DNN-based VQA methods are hindered by the limited scale of existing VQA datasets. As demonstrated in Tab. {{table:43c9402e-2914-42dc-99c4-b2424f71f6ad}} , public VQA datasets are significantly smaller in size when compared to video classification datasets {{cite:fbaab96}}, {{cite:dd352e3}}. The primary reason for this discrepancy is the prevale

In [89]:
# LOL, I don't actually have the title or abstract of the paper with the table! That's hilarious.
# I have to use s2 to get it!
def get_metadata_s2_public(corpus_ids_batch, prefix="CorpusId:"):
    base_url = "https://api.semanticscholar.org/graph/v1/paper/batch?fields=corpusId,isOpenAccess,openAccessPdf,externalIds,title,abstract"
    ids_json = {"ids": [f"{prefix}{corpus_id}" for corpus_id in corpus_ids_batch]}
    response = requests.post(base_url, json=ids_json)
    metadata = response.json()
    return metadata

In [90]:
arxiv_ids_batch = [ft['paper_id'] for ft in full_texts_2308_high_quality]
arxiv_ids_batch_stripped = [re.sub("v\d+", "", arxiv_id) for arxiv_id in arxiv_ids_batch]
metadata_batch = get_metadata_s2_public(arxiv_ids_batch_stripped, prefix="ARXIV:")

In [ ]:
with open("../data/arxiv_tables_2308_high_quality/full_text")

In [209]:
metadata_batch

[{'paperId': 'b5e4c46ac09d5de2051998c9f741ef480fab2ebe',
  'externalIds': {'DBLP': 'conf/mm/LiuWYSTZWL23',
   'ArXiv': '2308.00729',
   'DOI': '10.1145/3581783.3611795',
   'CorpusId': 260378902},
  'corpusId': 260378902,
  'title': 'Ada-DQA: Adaptive Diverse Quality-aware Feature Acquisition for Video Quality Assessment',
  'abstract': 'Video quality assessment (VQA) has attracted growing attention in recent years. While the great expense of annotating large-scale VQA datasets has become the main obstacle for current deep-learning methods. To surmount the constraint of insufficient training data, in this paper, we first consider the complete range of video distribution diversity (i.e. content, distortion, motion) and employ diverse pretrained models (e.g. architecture, pretext task, pre-training dataset) to benefit quality representation. An Adaptive Diverse Quality-aware feature Acquisition (Ada-DQA) framework is proposed to capture desired quality-related features generated by these

In [212]:
metadata_map = {md['externalIds']['ArXiv']: md for md in metadata_batch}

In [363]:
for md in metadata_batch:
    papers_2308_high_quality.append({
        'tabids': [],
        'corpus_id': md['externalIds']['CorpusId'],
        'arxiv_id': md['externalIds']['ArXiv'],
        'paper_id': md['paperId'],
        'title': md['title'],
        'abstract': md['abstract'],
    })
    
# papers_2308_high_quality[0]

In [136]:
# Ok, now we have the title and abstract, let'd combine this into a prompt
title = metadata_batch[0]["title"]
abstract = metadata_batch[0]["abstract"]
table_section = "\n".join(relevant_sections['Introduction'])

for scheme, glossary_key in glossary_keys:
    if scheme == glossary_key:
        query = f"In the context of the following table, what does '{glossary_key}' refer to?\n"
    else:
        query = f"In the context of the following table, what does the '{scheme}' '{glossary_key}' refer to?"
    table = str(pd.DataFrame(dataset_2308_high_quality_sample[0]['table_json']['table_dict'])) + "\n"
    table += "Caption: " + dataset_2308_high_quality_sample[0]['table_unfiltered']['caption']
    break

In [174]:
instruction = f"""\
Based on the following text from a scientific paper, answer the question about the table that follows:

Title: {title}

Abstract: {abstract}

Section: {list(relevant_sections.keys())[0]}
{table_section}

---
Based on the above text, in the context of the following table, what does 'Dataset' refer to? Answer in a single sentence.
Table: 43c9402e-2914-42dc-99c4-b2424f71f6ad
{table}\
"""

prompt = [{"role": "user", "content": instruction}]

In [145]:
print(instruction)

Based on the following text from a scientific paper, answer the question about the table that follows:

Title: Ada-DQA: Adaptive Diverse Quality-aware Feature Acquisition for Video Quality Assessment

Abstract: Video quality assessment (VQA) has attracted growing attention in recent years. While the great expense of annotating large-scale VQA datasets has become the main obstacle for current deep-learning methods. To surmount the constraint of insufficient training data, in this paper, we first consider the complete range of video distribution diversity (i.e. content, distortion, motion) and employ diverse pretrained models (e.g. architecture, pretext task, pre-training dataset) to benefit quality representation. An Adaptive Diverse Quality-aware feature Acquisition (Ada-DQA) framework is proposed to capture desired quality-related features generated by these frozen pretrained models. By leveraging the Quality-aware Acquisition Module (QAM), the framework is able to extract more essent

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

DEVICE = "cuda"
mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", cache_dir="/gscratch/xlab/blnewman/models/transformers/")
mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    cache_dir="/gscratch/xlab/blnewman/models/transformers/",
    load_in_8bit=True,
    # torch_dtype=torch.float16,
    # device_map=DEVICE,
)
mistral_model.config.pad_token_id = mistral_model.config.eos_token_id

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [164]:
torch.cuda.empty_cache()

In [166]:
mistral_model.get_memory_footprint()

8041021440

In [169]:
mistral_model.device

device(type='cuda', index=0)

In [170]:
inputs = mistral_tokenizer.apply_chat_template(prompt, return_tensors="pt").to(DEVICE)
generated_ids = mistral_model.generate(inputs, max_new_tokens=1000, do_sample=True, num_return_sequences=1)
mistral_tokenizer.batch_decode(generated_ids[:, inputs.shape[1]:], skip_special_tokens=True)

In [171]:
mistral_tokenizer.batch_decode(generated_ids[:, inputs.shape[1]:], skip_special_tokens=True)

["In the context of the table, 'Dataset' refers to the specific videocontent dataset mentioned in each row, for example, KoNViD-1k, LIVE-VQC, YouTube-UGC, LSVQ, KoNViD-150k, Sports-1M, and Kinetics-400. These datasets are used for different tasks, such as Video Quality Assessment (VQA) or video classification, and vary in size and number of annotations available."]

In [173]:
# "Answer in a single sentence"
inputs = mistral_tokenizer.apply_chat_template(prompt, return_tensors="pt").to(DEVICE)
generated_ids = mistral_model.generate(inputs, max_new_tokens=1000, do_sample=True, num_return_sequences=1)
mistral_tokenizer.batch_decode(generated_ids[:, inputs.shape[1]:], skip_special_tokens=True)

["In the context of the table in the text, 'Dataset' refers to a specific collection of video data along with associated annotations, used for the tasks of video quality assessment (VQA) or video classification."]

In [180]:
glossary_keys_unique = []
for key in glossary_keys:
    if key not in glossary_keys_unique:
        glossary_keys_unique.append(key)

In [196]:
table_df = pd.DataFrame(dataset_2308_high_quality_sample[0]['table_json']['table_dict'])

In [200]:
table_df[["References", "Task"]]

,References,Task
0,{{cite:24b4d04}},VQA
1,{{cite:8f3eb86}},VQA
2,{{cite:9be8207}},VQA
3,{{cite:3dfd446}},VQA
4,{{cite:9c04ab9}},VQA
5,{{cite:fbaab96}},classification
6,{{cite:dd352e3}},classification


In [38]:
# fill the whole glossary:
# table = str(pd.DataFrame(dataset_2308_high_quality_sample[0]['table_json']['table_dict'])) + "\n"
# table += "Caption: " + dataset_2308_high_quality_sample[0]['table_unfiltered']['caption']
def query_mistral(prompt):
    generated_ids = []
    inputs = mistral_tokenizer.apply_chat_template(prompt, return_tensors="pt").to(DEVICE)
    try:
        generated_ids = mistral_model.generate(inputs, max_new_tokens=1000, do_sample=True, num_return_sequences=1)
        response = mistral_tokenizer.batch_decode(generated_ids[:, inputs.shape[1]:], skip_special_tokens=True)
    except torch.cuda.OutOfMemoryError:
        print("oom:", inputs.shape, flush=True)
        raise torch.cuda.OutOfMemoryError
    finally:
        del inputs
        del generated_ids
        torch.cuda.empty_cache()

    return response

def create_glossary_instructions(title, abstract, section_name, table_section, glossary_query, table_hash, table):
        instruction = f"""\
Based on the following text from a scientific paper, answer the question about the table that follows:

Title: {title}

Abstract: {abstract}

Section: {section_name}
{table_section}

---
Based on the above text, in the context of the following table, what does {glossary_query} refer to? Answer in a single sentence. If the answer is not clear just write 'unanswerable'.
Table: {table_hash}
{table}\
"""
        
        return instruction

def create_glossary(dataset_table, glossary_keys, title, abstract, section_name, table_section):
    table_df = pd.DataFrame(dataset_table['table_json']['table_dict'])
    # glossary = {("Dataset", "Dataset"): "In the context of the table in the text, 'Dataset' refers to a specific collection of video data along with associated annotations, used for the tasks of video quality assessment (VQA) or video classification."}
    glossary = {}
    for scheme, glossary_key in glossary_keys:
        # Avoid repeating glossary terms
        if (scheme, glossary_key) in glossary:
            continue
        print((scheme, glossary_key), flush=True)
        if scheme == glossary_key:
            glossary_query = f"'{glossary_key}'"
        else:
            glossary_query = f"'{glossary_key}' in the column '{scheme}'"
    
        # mask out the columns that aren't being asked about:
        table = str(table_df[["References", scheme]]) + "\n"
        table += "Caption: " + dataset_table['table_unfiltered']['caption']

        instruction = create_glossary_instructions(
            title, abstract, section_name, table_section, glossary_query, dataset_table['_full_text_table_hash'], table
        )

        while True:
            prompt = [{"role": "user", "content": instruction}]
            try:
                response = query_mistral(prompt)
                break
            except torch.cuda.OutOfMemoryError:
                print("OOM num chars:", len(instruction))
                table_section = "\n".join(table_section.split("\n")[:-1])
                instruction = create_glossary_instructions(
                    title, abstract, section_name, table_section, glossary_query, dataset_table['_full_text_table_hash'], table
                )
        # inputs = mistral_tokenizer.apply_chat_template(prompt, return_tensors="pt").to(DEVICE)
        # try:
        #     # generated_ids = mistral_model.generate(inputs, max_new_tokens=1000, do_sample=True, num_return_sequences=1)
        #     response = query_mistral(prompt)
        # except torch.cuda.OutOfMemoryError:
        #     print("oom:", flush=True)
        #     # print("oom:", inputs.shape)
        #     # del inputs
        #     torch.cuda.empty_cache()
        #     new_table_section = "\n".join(table_section.split("\n")[:-1])
        #     return create_glossary(dataset_table, glossary_keys, title, abstract, section_name, new_table_section)
        #     # remove the last paragraph from the table_section and try again lol
        # response = mistral_tokenizer.batch_decode(generated_ids[:, inputs.shape[1]:], skip_special_tokens=True)
        # response = ["test"]
        glossary[(scheme, glossary_key)] = response[0]
        # del inputs
        # del generated_ids
        # torch.cuda.empty_cache()
    return instruction, glossary

In [9]:
def listify(list_paragraph):
        return " * " + "\n * ".join(list_paragraph.strip().split("\n\n"))

def get_section_with_table(dataset_table, full_text):
    section_names = []
    for paragraph in full_text['body_text']:
        for ref_span in paragraph['ref_spans']:
            # if "DNN-based VQA method" in paragraph["body_text"]:
            #     print(paragraph)
            if ref_span["ref_id"] == dataset_table['_full_text_table_hash']:
                # print(paragraph['content_type'])
                # print(paragraph.keys())
                section_names.append(paragraph['section'])
    
    relevant_sections = {sn: [] for sn in section_names}
    for paragraph in full_text['body_text']:
        if paragraph['section'] in relevant_sections:
            if paragraph['content_type'] == "paragraph":
                relevant_sections[paragraph['section']].append(paragraph['text'])
            elif paragraph['content_type'] == "list":
                relevant_sections[paragraph['section']].append(listify(paragraph['text']))
            # else:
            #     print(paragraph['content_type'])

    # just return the first section for now
    table_section = "\n".join(list(relevant_sections.values())[0])
    return list(relevant_sections.keys())[0], table_section

In [359]:
for dataset_table_i, dataset_table in enumerate(dataset_2308_high_quality_sample):
    print("\n----------\n")
    print(dataset_table_i, dataset_table["paper_id"], dataset_table["_full_text_table_hash"])
    full_text = full_texts_2308_high_quality_map[dataset_table['paper_id']]
    # first, create the unique glossary keys:
    glossary_keys = [
        (scheme.strip(), str(entry).strip())
        for scheme, column in
        dataset_table['table_json']['table_dict'].items()
        for entry in [scheme] + column
        if scheme != "References" and not is_numeric(str(entry)) and not is_na(str(entry)) and not is_binary(str(entry))
    ]
    glossary_keys_unique = []
    for key in glossary_keys:
        if key not in glossary_keys_unique:
            glossary_keys_unique.append(key)

    print(glossary_keys_unique)


----------

0 2308.00729v1 43c9402e-2914-42dc-99c4-b2424f71f6ad
[('Dataset', 'Dataset'), ('Dataset', 'KoNViD-1k'), ('Dataset', 'LIVE-VQC'), ('Dataset', 'YouTube-UGC'), ('Dataset', 'LSVQ'), ('Dataset', 'KoNViD-150k'), ('Dataset', 'Sports-1M'), ('Dataset', 'Kinetics-400'), ('Task', 'Task'), ('Task', 'VQA'), ('Task', 'classification'), ('Size', 'Size'), ('Annotations', 'Annotations'), ('Annotations', '- (auto.)')]

----------

1 2308.06173v1 8750716b-7f93-4922-8d97-ff903f750be7
[('Attack', 'Attack'), ('Attack', 'IPatch'), ('Attack', 'SSAttack'), ("Attacker's Knowledge", "Attacker's Knowledge"), ("Attacker's Knowledge", 'White-box'), ('Robustness Technique', 'Robustness Technique'), ('Robustness Technique', 'EOT'), ('Stealthiness Technique', 'Stealthiness Technique'), ('Physical test type', 'Physical test type'), ('Physical test type', 'Static'), ('Space', 'Space'), ('Space', '2D')]

----------

2 2308.07107v2 e9b07ad7-35e6-4776-8561-9658e198363e
[('Methods', 'Methods'), ('Methods', 'InPa

In [39]:
# contextualization_inputs = {}
# alright, let's put this in a loop to get all of the glossaries:
start_i = 26
for dataset_table_i, dataset_table in enumerate(dataset_2308_high_quality_sample[start_i:]):
    dataset_table_i += start_i
    print("\n----------\n")
    print(dataset_table_i, dataset_table["paper_id"], dataset_table["_full_text_table_hash"])
    full_text = full_texts_2308_high_quality_map[dataset_table['paper_id']]
    # first, create the unique glossary keys:
    glossary_keys = [
        (scheme.strip(), str(entry).strip())
        for scheme, column in
        dataset_table['table_json']['table_dict'].items()
        for entry in [scheme] + column
        if scheme != "References" and not is_numeric(str(entry)) and not is_na(str(entry)) and not is_binary(str(entry))
    ]
    # glossary_keys_unique = []
    # for key in glossary_keys:
    #     if key not in glossary_keys_unique:
    #         glossary_keys_unique.append(key)

    # next, get the section, title and abstract
    section_name, table_section = get_section_with_table(dataset_table, full_text)
    arxiv_id = re.sub("v\d+", "", dataset_table["paper_id"])
    title = metadata_map[arxiv_id]['title']
    abstract = metadata_map[arxiv_id]['abstract']
    print(section_name)
    # print(table_section)
    instruction_sample, glossary = create_glossary(dataset_table, glossary_keys, title, abstract, section_name, table_section)

    dataset_table["context_autogenerated"] = {
        "glossary": {"<~>".join(k): v for k, v in glossary.items()},
        "inputs": {
            "section_name": section_name,
            "instruction_sample": instruction_sample,
        }
    }

    dataset_table["title"] = title
    dataset_table["abstract"] = abstract
    break
    # "title": title,
    # "abstract": abstract,

    # contextualization_inputs[dataset_table["_table_hash"]] = {
    #     "glossary": {"<~>".join(k): v for k, v in glossary.items()},
    #     "inputs": {
    #         "title": title,
    #         "abstract": abstract,
    #         "section_name": section_name,
    #         "table_section": table_section,
    #         "instruction_sample": instruction_sample,
    #     }
    # }


----------

26 2308.15701v1 ef95492e-6dde-41e2-89b4-70db1dab2cf5
Methods in MBSR
('Works', 'Works')
oom: torch.Size([1, 17582])
OOM num chars: 54114
oom: torch.Size([1, 17581])
OOM num chars: 54113
oom: torch.Size([1, 17522])
OOM num chars: 53786
oom: torch.Size([1, 17371])
OOM num chars: 53232
oom: torch.Size([1, 17370])
OOM num chars: 53231
oom: torch.Size([1, 17350])
OOM num chars: 53108
oom: torch.Size([1, 17282])
OOM num chars: 52762
oom: torch.Size([1, 17281])
OOM num chars: 52761
oom: torch.Size([1, 17227])
OOM num chars: 52473
oom: torch.Size([1, 17161])
OOM num chars: 52158
oom: torch.Size([1, 17122])
OOM num chars: 51971
oom: torch.Size([1, 17047])
OOM num chars: 51656
oom: torch.Size([1, 17042])
OOM num chars: 51650
oom: torch.Size([1, 17041])
OOM num chars: 51649
oom: torch.Size([1, 16949])
OOM num chars: 51228
oom: torch.Size([1, 16931])
OOM num chars: 51148
oom: torch.Size([1, 16912])
OOM num chars: 51069
oom: torch.Size([1, 16911])
OOM num chars: 51068
oom: torch.Size([

Weird ones (thing isn't included): 2, 4, 5, 7, 8

In [41]:
with open("../data/arxiv_tables_2308_high_quality/dataset_autocontext.jsonl", "w") as f:
    for line in dataset_2308_high_quality_sample:
        f.write(json.dumps(line) + "\n")

# with open("../data/arxiv_tables_2308_high_quality/context_info.jsonl", "w") as f:
#     json.dump(contextualization_inputs, f)

In [370]:
dataset_2308_high_quality_sample[-1].keys()

dict_keys(['paper_id', '_pdf_hash', '_source_hash', '_source_name', '_table_hash', 'table_html', 'table_json', 'bib_hash', 'row_bib_map', 'table_unfiltered', '_table_hash_full_text', '_full_text_table_hash', 'context_autogenerated', 'title', 'abstract'])

In [40]:
for dti, dataset_table in enumerate(dataset_2308_high_quality_sample):
    # print(list(dataset_table["context_autogenerated"]["glossary"].values())[1])
    if list(dataset_table["context_autogenerated"]["glossary"].values())[1] == "test":
        print(dti)

In [42]:
dataset_2308_high_quality_sample[26]["context_autogenerated"]["glossary"]

{'Works<~>Works': "In the context of the table, 'Works' refers to the specific research works or models that have been mentioned in the text and are listed in the first column of the table, each with a corresponding reference.",
 'Works<~>RLBL': 'RLBL refers to the recurrent log-bilinear model for next-item recommendation as described in the given text.',
 'Works<~>RIB': 'RIB in the table refers to the interpretable recommendation framework from the micro behavior perspective (RIB), which models heterogeneous behaviors and dwell time to capture more fine-grained user information using GRU.',
 'Works<~>BINN': 'BINN in the context of the table refers to Behavior-intensive neural network, which is one of the works mentioned in the table that uses RNN learning architecture for multi-behavior sequential recommendation.',
 'Works<~>CBS': 'CBS stands for CBS model, which is mentioned as one of the works in the table that utilizes RNN learning architecture for multi-behavior sequential recomme

In [44]:
total = 0
for dataset_table in dataset_2308_high_quality_sample:
    total += len(dataset_table["context_autogenerated"]["glossary"])

In [45]:
total

624

In [50]:
dataset_2308_high_quality_sample[22]["table_unfiltered"]

{'table': '<table rend="inline"><row><cell right-border="true" halign="left">Defense Method</cell>\n<cell right-border="true" halign="left">Type</cell>\n<cell halign="left">Time Cost</cell>\n<cell>Description</cell>\n</row><row><cell right-border="true" halign="left">Standard</cell>\n<cell right-border="true" halign="left">4*Data augmentations</cell>\n<cell halign="left">Low</cell>\n<cell>Random image cropping and flipping</cell>\n</row><row><cell right-border="true" halign="left">CutOut\xa0<cit sha="401c9eff85fb4a8b8b4fdfa18023550fbebfded4"><ref target="bid24"/></cit>{{cite:401c9ef}}</cell>\n<cell right-border="true" halign="left"/>\n<cell halign="left">Low</cell>\n<cell>Random image erasing</cell>\n</row><row><cell right-border="true" halign="left">MixUp\xa0<cit sha="01545b6a8afa3b8e3be1dba5a5dba35ae9a68e34"><ref target="bid26"/></cit>{{cite:01545b6}}</cell>\n<cell right-border="true" halign="left"/>\n<cell halign="left">Low</cell>\n<cell>Random image blending</cell>\n</row><row><cel

In [43]:
dataset_2308_high_quality_sample[26]["table_json"]["table_dict"]

{'References': ['{{cite:bcf84d7}}',
  '{{cite:2f67c58}}',
  '{{cite:909f272}}',
  '{{cite:8dbb1e8}}',
  '{{cite:9641864}}',
  '{{cite:c76954f}}',
  '{{cite:82d42f0}}',
  '{{cite:b1e6c16}}',
  '{{cite:15341b1}}'],
 'Works': ['RLBL',
  'RIB',
  'BINN',
  'CBS',
  'DIPN',
  'HUP',
  'IARS',
  'DeepRec',
  'MBN'],
 'Data Perspective': ['A sequence of (item, behavior) pairs',
  'A sequence of (item, behavior) pairs',
  'A sequence of (item, behavior) pairs',
  'Some behavior-specific subsequences of items',
  'Some behavior-specific subsequences of items',
  'A sequence of (item, behavior) pairs',
  'A sequence of (item, behavior) pairs',
  'Some behavior-specific subsequences of items',
  'Some behavior-specific subsequences of items'],
 'Model Perspective': ['Local',
  'Local',
  'Local',
  'Local',
  'Local',
  'Local',
  'Local',
  'Local + Global',
  'Local'],
 'Features': ['Capture the influence of heterogeneous behaviors by utilizing a behavior transition matrix.',
  'Leverage GRU an

In [333]:
for table in dataset_table

17

In [336]:
contextualization_inputs.keys()

dict_keys(['2308.00729v1', '2308.06173v1', '2308.07107v2', '2308.00937v1', '2308.08407v1', '2308.06953v3', '2308.06966v1', '2308.14430v1', '2308.14371v2', '2308.09090v1', '2308.08794v1', '2308.03149v1', '2308.13266v2', '2308.08643v1', '2308.10631v2', '2308.02151v1', '2308.03108v1'])

In [346]:
list(contextualization_inputs.keys()).index('2308.09090v1')

9

In [341]:
print(contextualization_inputs['2308.09090v1']['inputs']['instruction_sample'])

Based on the following text from a scientific paper, answer the question about the table that follows:

Title: Data-driven Integrated Sensing and Communication: Recent Advances, Challenges, and Future Prospects

Abstract: Integrated Sensing and Communication (ISAC), combined with data-driven approaches, has emerged as a highly significant field, garnering considerable attention from academia and industry. Its potential to enable wide-scale applications in the future sixth-generation (6G) networks has led to extensive recent research efforts. Machine learning (ML) techniques, including $K$-nearest neighbors (KNN), support vector machines (SVM), deep learning (DL) architectures, and reinforcement learning (RL) algorithms, have been deployed to address various design aspects of ISAC and its diverse applications. Therefore, this paper aims to explore integrating various ML techniques into ISAC systems, covering various applications. These applications span intelligent vehicular networks, e

In [358]:
[t for t in dataset_2308_high_quality_sample if t['paper_id'] == '2308.07107v2'][0]['table_json']['table_dict'].keys()

dict_keys(['References', 'Methods', '# Examples', 'Generator', 'Synthetic Data', 'Filter Method', "LLMs' tuning"])

In [355]:
contextualization_inputs['2308.09090v1']['glossary'].keys()

dict_keys(['Dataset<~>Dataset', 'Technology<~>Technology', 'Technology<~>IRS-assisted V2I', 'Technology<~>General applications', 'Technology<~>IRS-assisted ISAC MISO', 'AI technique<~>AI technique', 'AI technique<~>DL', 'Criteria<~>Criteria', 'Criteria<~>CAP-Net (conv layers and LSTM)', 'Criteria<~>CNN (Trained as a denoiser)', 'Criteria<~>1.Direct estimation CNN (DE-CNN)2.Reflected estimation CNN (RE-CNN)', 'Input<~>Input', 'Input<~>Historical covariance of the received echo', 'Input<~>Channel estimated by LS method', 'Input<~>1.Received direct signals2.Total received signals and DE-CNN estimation', 'Output<~>Output', 'Output<~>AoAs', 'Output<~>Channel estimation', 'Output<~>Direct (DE-CNN) and reflected (RE-CNN) sensing and communication channels'])

In [357]:
contextualization_inputs['2308.07107v2']['glossary'].keys()

dict_keys(['Dataset<~>Dataset', 'Methods<~>Methods', 'Methods<~>REALM', 'Methods<~>RAG', 'Methods<~>REPLUG', 'Methods<~>Atlas', 'Methods<~>RETA-LLM', 'Methods<~>RALM', 'Methods<~>RETRO', 'Methods<~>IRCoT', 'Methods<~>FLARE', 'Backbone models<~>Backbone models', 'Backbone models<~>BERT', 'Backbone models<~>BART', 'Backbone models<~>GPT', 'Backbone models<~>T5', 'Backbone models<~>Gopher', 'Backbone models<~>LLaMA & GLM & GPT', 'Backbone models<~>LLaMA & OPT & GPT', 'Backbone models<~>Transformer', 'Backbone models<~>Flan-T5 & GPT', 'Where to incorporate retrieval<~>Where to incorporate retrieval', 'Where to incorporate retrieval<~>Input layer', 'Where to incorporate retrieval<~>Attention layer', 'When to retrieve<~>When to retrieve', 'When to retrieve<~>In the beginning', 'When to retrieve<~>During generation (every n tokens)', 'When to retrieve<~>During generation (every sentence)', 'When to retrieve<~>During generation (aperiodic)', 'How to use LLMs<~>How to use LLMs', 'How to use LLM

In [345]:
print(json.dumps(contextualization_inputs['2308.09090v1']['glossary'], indent=2))

{
  "Dataset<~>Dataset": "In the context of the table in the text, 'Dataset' refers to a specific collection of video data along with associated annotations, used for the tasks of video quality assessment (VQA) or video classification.",
  "Technology<~>Technology": "In the context of the table, 'Technology' refers to the specific application domain of the DL-ISAC techniques used for channel estimation and IRS, such as IRS-assisted V2I, general applications, and IRS-assisted ISAC MISO.",
  "Technology<~>IRS-assisted V2I": "IRS-assisted V2I refers to Intelligent Reflecting Surface (IRS) assisted vehicle-to-infrastructure (V2I) communication systems.",
  "Technology<~>General applications": "'General applications' in the column 'Technology' refers to applications of deep learning in ISAC systems beyond Intelligent Reflecting Surface (IRS)-assisted ISAC and V2I communications.",
  "Technology<~>IRS-assisted ISAC MISO": "IRS-assisted ISAC MISO refers to Integrated Sensing and Communication

In [216]:
dataset_2308_high_quality_sample[0].keys()

dict_keys(['paper_id', '_pdf_hash', '_source_hash', '_source_name', '_table_hash', 'table_html', 'table_json', 'bib_hash', 'row_bib_map', 'table_unfiltered', '_table_hash_full_text', 'context', '_full_text_table_hash'])

In [210]:
metadata_map = {md['ArXiv']: md for md in metadata_batch}

{'paperId': 'b5e4c46ac09d5de2051998c9f741ef480fab2ebe',
 'externalIds': {'DBLP': 'conf/mm/LiuWYSTZWL23',
  'ArXiv': '2308.00729',
  'DOI': '10.1145/3581783.3611795',
  'CorpusId': 260378902},
 'corpusId': 260378902,
 'title': 'Ada-DQA: Adaptive Diverse Quality-aware Feature Acquisition for Video Quality Assessment',
 'abstract': 'Video quality assessment (VQA) has attracted growing attention in recent years. While the great expense of annotating large-scale VQA datasets has become the main obstacle for current deep-learning methods. To surmount the constraint of insufficient training data, in this paper, we first consider the complete range of video distribution diversity (i.e. content, distortion, motion) and employ diverse pretrained models (e.g. architecture, pretext task, pre-training dataset) to benefit quality representation. An Adaptive Diverse Quality-aware feature Acquisition (Ada-DQA) framework is proposed to capture desired quality-related features generated by these frozen 

In [207]:
glossary_v3 = create_glossary(dataset_2308_high_quality_sample[0])

('Dataset', 'KoNViD-1k')
('Dataset', 'LIVE-VQC')
('Dataset', 'YouTube-UGC')
('Dataset', 'LSVQ')
('Dataset', 'KoNViD-150k')
('Dataset', 'Sports-1M')
('Dataset', 'Kinetics-400')
('Task', 'Task')
('Task', 'VQA')
('Task', 'classification')
('Size', 'Size')
('Annotations', 'Annotations')
('Annotations', '- (auto.)')


In [202]:
glossary

{('Dataset',
  'Dataset'): "In the context of the table in the text, 'Dataset' refers to a specific collection of video data along with associated annotations, used for the tasks of video quality assessment (VQA) or video classification.",
 ('Dataset',
  'KoNViD-1k'): 'KoNViD-1k is a publicly available no-reference video quality assessment dataset that consists of 1000 videos for training and testing purposes.',
 ('Dataset',
  'LIVE-VQC'): 'LIVE-VQC is a mainstream high-quality annotated Non-Reference Video Quality Assessment (NR-VQA) dataset.',
 ('Dataset',
  'YouTube-UGC'): 'YouTube-UGC in the table refers to a specific dataset named "YouTube-UGC" used in the context of video quality assessment.',
 ('Dataset',
  'LSVQ'): 'LSVQ in the given table refers to a public VQA dataset named Large Scale Video Quality (LSVQ) dataset.',
 ('Dataset',
  'KoNViD-150k'): 'KoNViD-150k is a large-scale video quality assessment dataset referenced in the text.',
 ('Dataset',
  'Sports-1M'): 'Sports-1M i

In [195]:
# ask for "unanswerable", "single sentence"
glossary_v2 = {k: v for k, v in glossary.items()}

In [192]:
glossary_v1

{('Dataset',
  'Dataset'): "In the context of the table in the text, 'Dataset' refers to a specific collection of video data along with associated annotations, used for the tasks of video quality assessment (VQA) or video classification.",
 ('Dataset',
  'KoNViD-1k'): 'KoNViD-1k is a publicly available VQA dataset with 1,200 videos and an average of 114 subjective quality scores per video.',
 ('Dataset',
  'LIVE-VQC'): 'LIVE-VQC is a publicly available VQA dataset with 585 videos and 240 annotations per video.',
 ('Dataset',
  'YouTube-UGC'): 'YouTube-UGC in the table refers to the YouTube Unlabelled Generic Video Dataset used for VQA tasks.',
 ('Dataset',
  'LSVQ'): "LSVQ in the column 'Dataset' refers to a publicly available Video Quality Assessment dataset with a size of 39,075 videos and 35 annotations per video.",
 ('Dataset',
  'KoNViD-150k'): 'KoNViD-150k is a VQA dataset mentioned in the text, which contains 153,841 videos.',
 ('Dataset',
  'Sports-1M'): "'Sports-1M' in the col

In [ ]:
# this is ok... but it relies a bit too much on the table... I wonder if we could somehow subset the table and if that would be better?

In [188]:
print(instruction)

Based on the following text from a scientific paper, answer the question about the table that follows:

Title: Ada-DQA: Adaptive Diverse Quality-aware Feature Acquisition for Video Quality Assessment

Abstract: Video quality assessment (VQA) has attracted growing attention in recent years. While the great expense of annotating large-scale VQA datasets has become the main obstacle for current deep-learning methods. To surmount the constraint of insufficient training data, in this paper, we first consider the complete range of video distribution diversity (i.e. content, distortion, motion) and employ diverse pretrained models (e.g. architecture, pretext task, pre-training dataset) to benefit quality representation. An Adaptive Diverse Quality-aware feature Acquisition (Ada-DQA) framework is proposed to capture desired quality-related features generated by these frozen pretrained models. By leveraging the Quality-aware Acquisition Module (QAM), the framework is able to extract more essent

In [184]:
glossary

{('Dataset',
  'Dataset'): "In the context of the table in the text, 'Dataset' refers to a specific collection of video data along with associated annotations, used for the tasks of video quality assessment (VQA) or video classification.",
 ('Dataset', 'KoNViD-1k'): 'test',
 ('Dataset', 'LIVE-VQC'): 'test',
 ('Dataset', 'YouTube-UGC'): 'test',
 ('Dataset', 'LSVQ'): 'test',
 ('Dataset', 'KoNViD-150k'): 'test',
 ('Dataset', 'Sports-1M'): 'test',
 ('Dataset', 'Kinetics-400'): 'test',
 ('Task', 'Task'): 'test',
 ('Task', 'VQA'): 'test',
 ('Task', 'classification'): 'test',
 ('Size', 'Size'): 'test',
 ('Annotations', 'Annotations'): 'test',
 ('Annotations', '- (auto.)'): 'test'}

In [150]:
inputs.shape

torch.Size([1, 2483])

In [152]:
mistral_model.device

device(type='cuda', index=0)

In [62]:
dataset_2308_high_quality_sample[0]['_full_text_table_hash']

'24da7cab-1a2b-4e9c-b0e0-fa3182383d36'

In [65]:
dataset_2308_high_quality_sample[0]['paper_id']

'2308.00729v1'

In [66]:
full_text['paper_id']

'2308.00729v1'

In [77]:
papers_2308_high_quality[0]

{'tabids': ['b42af37c-c83f-4cc0-8cc8-e7221c6a8d0f'],
 'corpus_id': 9136312,
 'title': 'The Konstanz natural video database (KoNViD-1k)',
 'paper_id': 'ae95abd2406ddfab1aa2ffa2413d68b98b1ba21b',
 'abstract': 'Subjective video quality assessment (VQA) strongly depends on semantics, context, and the types of visual distortions. Currently, all existing VQA databases include only a small number of video sequences with artificial distortions. The development and evaluation of objective quality assessment methods would benefit from having larger datasets of real-world video sequences with corresponding subjective mean opinion scores (MOS), in particular for deep learning purposes. In addition, the training and validation of any VQA method intended to be ‘general purpose’ requires a large dataset of video sequences that are representative of the whole spectrum of available video content and all types of distortions. We report our work on KoNViD-1k, a subjectively annotated VQA database consist